In [12]:
import numpy as np
from mesa import Agent, Model
from mesa.time import RandomActivation

In [8]:
class MoneyAgent(Agent):
    '''initial wealth'''

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth  = 1

class MoneyModel(Model):
    '''model with some number of agents'''

    def __init__(self, N):
        self.num_agents = N
        # Create Agents
        for i in range(self.num_agents): 
            a = MoneyAgent(i, self)
            #print('Hi ! I am agent number ', i )
            self.schedule.add(a)

    def step(self):
        '''advanced the model by one step'''
        self.schedule.step()

In [11]:
x = MoneyModel(5)
x.num_agents

Hi ! I am agent number  0
Hi ! I am agent number  1
Hi ! I am agent number  2
Hi ! I am agent number  3
Hi ! I am agent number  4


5